In [2]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

Saving kaggle.json to kaggle.json
401 - Unauthorized - Unauthenticated


In [3]:
!kaggle datasets download -d armaanoajay/ccmt-crop-pest-and-disease-detection

Dataset URL: https://www.kaggle.com/datasets/armaanoajay/ccmt-crop-pest-and-disease-detection
License(s): unknown
100% 4.99G/5.00G [00:51<00:00, 194MB/s]
100% 5.00G/5.00G [00:51<00:00, 105MB/s]


In [4]:
!unzip '/content/ccmt-crop-pest-and-disease-detection.zip'

Streaming output truncated to the last 5000 lines.
  inflating: split-new/valid/anthracnose-cashew/11.png  
  inflating: split-new/valid/anthracnose-cashew/110.png  
  inflating: split-new/valid/anthracnose-cashew/111.png  
  inflating: split-new/valid/anthracnose-cashew/112.png  
  inflating: split-new/valid/anthracnose-cashew/113.png  
  inflating: split-new/valid/anthracnose-cashew/114.png  
  inflating: split-new/valid/anthracnose-cashew/115.png  
  inflating: split-new/valid/anthracnose-cashew/116.png  
  inflating: split-new/valid/anthracnose-cashew/117.png  
  inflating: split-new/valid/anthracnose-cashew/118.png  
  inflating: split-new/valid/anthracnose-cashew/119.png  
  inflating: split-new/valid/anthracnose-cashew/12.png  
  inflating: split-new/valid/anthracnose-cashew/120.png  
  inflating: split-new/valid/anthracnose-cashew/121.png  
  inflating: split-new/valid/anthracnose-cashew/122.png  
  inflating: split-new/valid/anthracnose-cashew/123.png  
  inflating: split-new/

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

# Paths to dataset directories
train_dir = 'split-new/train'
valid_dir = 'split-new/valid'
test_dir = 'split-new/test'

# ImageDataGenerator with enhanced data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,  # Increase rotation range
    zoom_range=0.2,     # Increase zoom range
    width_shift_range=0.25,
    height_shift_range=0.25,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Only rescale validation and test data
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load training, validation, and test datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Build a deeper and more complex CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')  # Output layer
])

# Compile the model with learning rate scheduling
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping and learning rate reduction on plateau
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // valid_generator.batch_size,
    epochs=50,  # Allow more epochs
    callbacks=[early_stopping, lr_scheduler]
)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

# Save the model for later use
model.save('ccmt5gb.h5')

Found 15062 images belonging to 22 classes.
Found 5012 images belonging to 22 classes.
Found 5052 images belonging to 22 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


102/470 ━━━━━━━━━━━━━━━━━━━━ 16:37 3s/step - accuracy: 0.0949 - loss: 5.5832

acc
